In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver
import numpy as np

In [2]:
carmax_url = 'https://www.carmax.com/cars/ford'
#carmax_url = 'https://www.carmax.com/'
#carmax_url = 'https://www.carvana.com/cars/ford'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [3]:
client = MongoClient('localhost', 27017)
db = client['capstone1']
pages = db['Ford_TX']

In [4]:
#Ford_files=['Ford_2013_2016.html','Ford_2017_2020.html']
#Ford_files = ['Ford_NY.html']
Ford_files = ['Ford_2009_2016_CA.html','Ford_2017_2021_CA.html']

In [5]:
url_links = []
for file in Ford_files:
    soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/" + file), "html.parser")
    for idx,link in enumerate(soup.find_all('article','car-tile')):
        url_links.append(link.a['href'])
        

In [6]:
car_mileage = []
car_price = []
car_mpg = []
car_model_year = []
car_model = []
car_make = []
car_color = []
car_transmission = []
car_engine_size = []
car_horse_power = []
car_engine_cylinder = []
car_msrp_price = []

url_links = list(set(url_links))

In [7]:
i=0

for link in url_links[301:]:
    scrape_url = link
    
    print(link,i)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    
    r_1 = requests.get(scrape_url,headers=headers)
    
    soup = BeautifulSoup(r_1.text,'html.parser')
    
    try:
        car_mileage.append(soup.find_all('div','value')[1].text.strip())
    except Exception as e:
        car_mileage.append(np.nan)
        
    try:    
        car_price.append(soup.find_all('div','value')[0].text.strip())
    except Exception as e:
        car_price.append(np.nan)
    
    try:  
        car_mpg.append([np.nan if soup.find_all('div','mpg')==[] else soup.find('div','mpg').text])
    except Exception as e:
        car_mpg.append(np.nan)
        
    try:
        car_model_year.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[0])
    except Exception as e:
        car_model_year.append(np.nan)
        
    try:
        car_model.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[1])
    except Exception as e:
        car_model.append(np.nan)
    
    try:
        car_make.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__model-trim').text.strip())
    except Exception as e:
        car_make.append(np.nan)
    
    a = soup.find_all('script')[6]
    a = str(a)
    
    try:
        car_color.append(a.split('ExteriorColor":"')[1].split('"')[0])
    except Exception as e:
        car_color.append(np.nan)
    
    try:
        car_transmission.append(a.split('Transmission":"')[1].split('"')[0])
    except Exception as e:
        car_transmission.append(np.nan)
    
    try:
        car_engine_size.append(float(a.split('EngineSize":"')[1].split('L')[0]))
    except Exception as e:
        car_engine_size.append(np.nan)
    
    
    
    if a.split('Horsepower":')[1].split(',')[0] != 'null':
        var_HorsePower = float(a.split('Horsepower":')[1].split(',')[0])
    else:
        var_HorsePower = np.nan  
    car_horse_power.append(var_HorsePower)
    
    
    if a.split('EngineCylinders":')[1].split(',')[0] != 'null':
        car_EngineCylinder = int(a.split('EngineCylinders":')[1].split(',')[0])
    else:
        car_EngineCylinder = np.nan
    car_engine_cylinder.append(car_EngineCylinder)
    
    try:
        car_msrp_price.append(float(a.split('Msrp":')[1].split(',')[0]))
    except Exception as e:
        car_msrp_price.append(np.nan)
    
#    pages.insert_one({'car_mileage': car_mileage, 
#                      'car_price': car_price, 
#                      'car_mpg': car_mpg ,
#                      'car_model_year': car_model_year,
#                      'car_model': car_model ,
#                      'car_make': car_make,
#                      'car_color': car_color,
#                      'car_transmission': car_transmission,
#                      'car_engine_size': car_engine_size,
#                      'car_horse_power': car_horse_power ,
#                      'car_engine_cylinder': car_engine_cylinder,
#                      'car_msrp_price':car_msrp_price})
    
    
    i += 1
    sleep(7)
    #if i>200:
    #    break 

https://www.carmax.com/car/20180804 0
https://www.carmax.com/car/19450120 1
https://www.carmax.com/car/20011178 2
https://www.carmax.com/car/20132121 3
https://www.carmax.com/car/19772587 4
https://www.carmax.com/car/19996348 5
https://www.carmax.com/car/19438184 6
https://www.carmax.com/car/19069846 7
https://www.carmax.com/car/19200411 8
https://www.carmax.com/car/19772843 9
https://www.carmax.com/car/20109276 10
https://www.carmax.com/car/20152829 11
https://www.carmax.com/car/20110784 12
https://www.carmax.com/car/20090743 13
https://www.carmax.com/car/19704733 14
https://www.carmax.com/car/19512047 15
https://www.carmax.com/car/19968844 16
https://www.carmax.com/car/20025194 17
https://www.carmax.com/car/19837724 18
https://www.carmax.com/car/19890833 19
https://www.carmax.com/car/20011603 20
https://www.carmax.com/car/19902246 21
https://www.carmax.com/car/20021263 22
https://www.carmax.com/car/19946205 23
https://www.carmax.com/car/19549645 24
https://www.carmax.com/car/20171365

https://www.carmax.com/car/19866456 208
https://www.carmax.com/car/20038342 209
https://www.carmax.com/car/19801832 210
https://www.carmax.com/car/20020460 211
https://www.carmax.com/car/19923915 212
https://www.carmax.com/car/19390108 213
https://www.carmax.com/car/19910001 214
https://www.carmax.com/car/19722084 215
https://www.carmax.com/car/19452803 216
https://www.carmax.com/car/20036768 217
https://www.carmax.com/car/19995782 218
https://www.carmax.com/car/20031457 219
https://www.carmax.com/car/19921797 220
https://www.carmax.com/car/19338900 221
https://www.carmax.com/car/19978506 222
https://www.carmax.com/car/19679556 223
https://www.carmax.com/car/19712630 224
https://www.carmax.com/car/19878540 225
https://www.carmax.com/car/19948232 226
https://www.carmax.com/car/19978462 227
https://www.carmax.com/car/20109636 228
https://www.carmax.com/car/20090361 229
https://www.carmax.com/car/20153440 230
https://www.carmax.com/car/19147316 231
https://www.carmax.com/car/20021700 232


In [8]:
df = pd.DataFrame({'car_mileage': car_mileage, 
                   'car_price': car_price, 
                   'car_mpg': car_mpg ,
                   'car_model_year': car_model_year,
                   'car_model': car_model ,
                   'car_make': car_make,
                   'car_color': car_color,
                   'car_transmission': car_transmission,
                   'car_engine_size': car_engine_size,
                   'car_horse_power': car_horse_power ,
                   'car_engine_cylinder': car_engine_cylinder,
                   'car_msrp_price':car_msrp_price,
                                          })
df.tail()

,car_mileage,car_price,car_mpg,car_model_year,car_model,car_make,car_color,car_transmission,car_engine_size,car_horse_power,car_engine_cylinder,car_msrp_price
295,31K miles,"$16,998*",[23/34 mpg],2019,Fusion SEL,Ford,Black,Automatic,1.5,181.0,4.0,28600.0
296,23K miles,"$15,998*",[23/30 mpg],2018,Escape SE,Ford,Gold,Automatic,1.5,179.0,4.0,25700.0
297,23K miles,"$15,998*",[15/22 mpg],2018,Escape S,Ford,White,Automatic,2.5,168.0,4.0,24000.0
298,102K miles,"$9,998*",[22/34 mpg],2013,Fusion S,Ford,Red,Automatic,2.5,170.0,4.0,NaN
299,16K miles,N/A*,[nan],2020,Explorer ST,Ford,Blue,Automatic,3.0,NaN,6.0,NaN


In [9]:
df.to_csv("Ford_CA_1.csv")

In [ ]:
#xxx= pd.read_csv('Ford_TX.csv')

In [ ]:
#xxx